In [ ]:


def remove_outliers(df,feature_list):
    ''' utilizes IQR to remove data which lies beyond 
    three standard deviations of the mean
    '''
    for feature in feature_list:
    
        #define interquartile range
        Q1= df[feature].quantile(0.25)
        Q3 = df[feature].quantile(0.75)
        IQR = Q3 - Q1
        #Set limits
        upper_limit = Q3 + 3 * IQR
        lower_limit = Q1 - 3 * IQR
        #remove outliers
        df = df[(df[feature] > lower_limit) & (df[feature] < upper_limit)]

    return df

def fill_null_with_corresponding_value(df, col1, col2):
    """
    Fills null values in column2 with corresponding non-null values from column1 in a pandas DataFrame
    """
    df_copy = df.copy()  # make a copy of the original dataframe to avoid modifying it
    df_copy[col2].fillna(df_copy[col1], inplace=True)  # fill null values in column2 with corresponding values from column1
    return df_copy

def mean_impute_columns(df, column_names):
    """
    Applies mode imputation to fill null values in specific columns of a pandas DataFrame
    """
    for column_name in column_names:
        mean_value = df[column_name].mean()
        df[column_name].fillna(mean_value, inplace=True)
    return df

def mode_impute_columns(df, column_names):
    """
    Applies mode imputation to fill null values in specific columns of a pandas DataFrame
    """
    for column_name in column_names:
        mode_value = df[column_name].mode().iloc[0]
        df[column_name].fillna(mode_value, inplace=True)
    return df

def convert_to_object_dtype(df, columns):
    """
    Convert specified columns in a pandas DataFrame to object dtype
    """
    for col in columns:
        if col in df.columns:
            df[col] = df[col].astype('object')
    return df


def clean_zillow(df):
    '''
    
    '''
    # isolate property type as single family
    df = df[df.propertylandusetypeid == (261 or 279)]
    # remove multi unit properties
    df = df[df.unitcnt == 1]
    # identify features to remove outliers
    feature_list = ['bathroomcnt', 'bedroomcnt', 'calculatedbathnbr', 'calculatedfinishedsquarefeet',
               'finishedsquarefeet12', 'fullbathcnt', 'lotsizesquarefeet', 'rawcensustractandblock', 
               'roomcnt', 'unitcnt', 'yearbuilt','assessmentyear','landtaxvaluedollarcnt', 
               'taxamount', 'censustractandblock']
    # rmove outliers
    remove_outliers(df,feature_list)
    # call function to fill nulls
    df = fill_null_with_corresponding_value(df, col1='rawcensustractandblock', col2='censustractandblock')
    # identify redundant or unuseful cols
    cols_to_drop = ['id','parcelid','calculatedbathnbr','finishedsquarefeet12',
                'fullbathcnt','unitcnt','rawcensustractandblock','propertyzoningdesc',
                'regionidcity','heatingorsystemtypeid', 'propertylandusetypeid', 
                'regionidzip', 'assessmentyear', 'propertylandusedesc', 
                'regionidcounty', 'roomcnt']
    # drop them
    df = df.drop(columns = cols_to_drop)
    # identify cols to impute nulls with mean
    cols_to_impute_mean = ['lotsizesquarefeet','structuretaxvaluedollarcnt', 'yearbuilt']
    # impute them
    df = mean_impute_columns(df, column_names=cols_to_impute_mean)
    # identify cols to impute with mode
    cols_to_impute_mode = ['buildingqualitytypeid','heatingorsystemdesc']
    # impute them
    df = mode_impute_columns(df, column_names=cols_to_impute_mode)
    # dop remaining nulls (2 of them)
    df = df.dropna()
    # identify cols we want as objects
    cols_to_obj = ['fips','hvac_type', 'propertylandusedesc','buildingqualitytypeid']
    # convert them by calling function
    df = convert_to_object_dtype(df, columns = cols_to_obj)
    #rename cols to promote readability
    df = df.rename(columns={'bathroomcnt': 'bathrooms', 'bedroomcnt': 'bedrooms', 
                        'buildingqualitytypeid': 'quality_id', 'calculatedfinishedsquarefeet': 'sqft',
                        'fips': 'county', 'lotsizesquarefeet': 'lot_sqft',
                        'propertycountylandusecode': 'prop_use_code', 'yearbuilt': 'year_built',
                        'structuretaxvaluedollarcnt': 'structure_value', 'taxvaluedollarcnt': 'home_value',
                        'censustractandblock': 'tract&block', 'transactiondate': 'txn_date',
                        'heatingorsystemdesc': 'hvac_type', 'landtaxvaluedollarcnt': 'land_value'})
    # convert fip code to county name
    df['county'] = df['county'].replace({6037.0: 'los_angeles', 6059.0: 'orange', 6111.0: 'ventura'})
    
    return df


def split_zillow(df):
    '''
    split_zillow will take in a single pandas df referencing a cleaned
    version of zillow data, and will then split the data into train,
    validate, and test sets stratifying on home_value
    
    Arguments: df. a pandas dataframe
    return: train, validate, test: the pandas df split from orginal df 
    '''
    train_val, test = train_test_split(df, random_state = 828, train_size = 0.8)
    train, validate = train_test_split(train_val, random_state = 828, train_size = 0.7)
    return train, validate, test


def scale_zillow(df):
    '''
    
    '''
    #identify features to scale
    feats_to_scale = ['bathrooms', 'bedrooms', 'sqft','lot_sqft', 'year_built',
                  'structure_value', 'home_value', 'land_value', 'taxamount']
    #for loop to name new scaled cols
    cols_scaled = [col + '_scaled' for col in feats_to_scale]
    # scale and transform
    df[cols_scaled] = MinMaxScaler().fit_transform(df[feats_to_scale])
    
    return df
    

  